In [1]:
import csv
import numpy as np
import pandas as pd
from math import log
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("train_feature.csv")
df

In [ ]:
sns.heatmap(df.isnull(),cmap='BuPu')

In [ ]:
corr=df.corr()
plt.figure(figsize=(20, 10))
sns.heatmap(corr,cmap='GnBu',annot=True,)

In [ ]:
df.info()

In [2]:
# 读文件获取训练数据集
# dataSet:数据集；features:特征名称；lable:标签集
def getDataSet():
    X = []
    with open('train_feature.csv', 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        print(header) #属性名称，数据第一行
        for row in reader:
            X.append(row)
    dataSet = np.array(X)
    lables=np.array(header)
    # 规范化数据集和特征
    lable=dataSet[:,5]
    dataSet = np.delete(dataSet, 5, 1)
    dataSet = np.insert(dataSet, 15, values=lable, axis=1)
    features = np.delete(lables, 5)
    lable = np.array(lable)
    return dataSet,features,lable

In [3]:
dataSet,features,lable=getDataSet()

['ip', 'app', 'device', 'os', 'channel', 'is_attributed', 'day', 'hour', 'minute', 'ip_count', 'app_count', 'device_count', 'os_count', 'channel_count', 'hour_count', 'minute_count']


In [ ]:
dataSet

In [ ]:
features

In [ ]:
lable.shape

In [4]:
def computeEntropy(dataSet):
    total = len(dataSet)
    lableCount = {}
    for sample in dataSet:
        lable = sample[-1]
        if lable not in lableCount:
            lableCount[lable]=1
        else:
            lableCount[lable]+=1
    Entropy = 0
    for key in lableCount:
        P = float(lableCount[key]/total)
        Entropy -= P*log(P,2)
    return Entropy

In [ ]:
dataSet[0:5]

In [ ]:
test=computeEntropy(dataSet[0:7000000])
test

In [ ]:
df.groupby(by="is_attributed").count()

In [5]:
# 找到可以用于分支的最佳特征，即使得信息增益最大的特征值
def findBestFeature(dataSet,features):
    maxGain = 0
    newEntropy = 0
    bestFeature = -1  # 最大熵增的特征,在特征列表中的索引
    oldEntropy = computeEntropy(dataSet)     #全局的熵值
    for i in range(len(features)):
        unique_feature_values = set(dataSet[:,i])   #去除重复值
        for value in unique_feature_values:
            sub_value_set = np.array([])
            # 提取出某一特征值的数据集,   此处有没有问题？
            for sample in dataSet:
                if sample[i] == value:
                    sub_value_set = np.append(sub_value_set,sample)
            # 计算该数据集的熵值
            subEntropy = computeEntropy(sub_value_set)
            subP = len(sub_value_set)/len(dataSet)
            newEntropy += subP*subEntropy
        gain = oldEntropy-newEntropy
        if gain > maxGain:
            maxGain = gain
            bestFeature = i
    return bestFeature

In [ ]:
bestFeature=findBestFeature(dataSet,features)
bestFeature